Задача: научиться обучать простейшую нейросетевую модель, практически эквивалентную алгоритму линейной регрессии, с помощью метода градиентного спуска в pytorch

https://www.kaggle.com/datasets/camnugent/california-housing-prices/code?datasetId=5227&searchQuery=NN

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [148]:
df = pd.read_csv('housing.csv')

In [149]:
df = df.dropna().drop('ocean_proximity', axis=1)

In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20433 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
dtypes: float64(9)
memory usage: 1.6 MB


In [151]:
features = df.drop("median_house_value", axis=1)
labels = df["median_house_value"]

In [152]:
scaler = StandardScaler()
features = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)

In [153]:
features.values

array([[-1.32731375,  1.05171726,  0.98216331, ..., -0.97331952,
        -0.97683327,  2.34516291],
       [-1.32232256,  1.04235526, -0.60621017, ...,  0.86133924,
         1.67037262,  2.33263161],
       [-1.33230494,  1.03767426,  1.85576873, ..., -0.81976943,
        -0.84342665,  1.78293943],
       ...,
       [-0.82320322,  1.77727236, -0.92388486, ..., -0.36882637,
        -0.17377773, -1.14317103],
       [-0.87311515,  1.77727236, -0.84446619, ..., -0.60356386,
        -0.39350628, -1.05513604],
       [-0.83318561,  1.74918635, -1.00330353, ..., -0.03348711,
         0.07995643, -0.78060586]])

In [155]:
X_np = features.to_numpy()
y_np = labels.to_numpy()

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np/1000, test_size=0.2, random_state=42)

In [157]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16346, 8), (4087, 8), (16346,), (4087,))

In [158]:
y_train.shape = (y_train.shape[0], 1)
y_test.shape = (y_test.shape[0], 1)

In [159]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16346, 8), (4087, 8), (16346, 1), (4087, 1))

In [160]:
class LinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.linear(x)
        return out

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

input_size = X_train.shape[1]
output_size = 1
learning_rate = 0.001
num_epochs = 100

model = LinearRegression(input_size, output_size)

criterion = nn.MSELoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


for epoch in range(num_epochs):

    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Эпоха [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Параметры модели:')
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f'{name}: {param.data.numpy()}')

Эпоха [10/100], Loss: 54267.4648
Эпоха [20/100], Loss: 52383.4883
Эпоха [30/100], Loss: 50573.6680
Эпоха [40/100], Loss: 48835.0391
Эпоха [50/100], Loss: 47164.7148
Эпоха [60/100], Loss: 45559.9766
Эпоха [70/100], Loss: 44018.1992
Эпоха [80/100], Loss: 42536.8672
Эпоха [90/100], Loss: 41113.5742
Эпоха [100/100], Loss: 39746.0195
Параметры модели:
linear.weight: [[-1.3555669 -2.8468208  2.3197265  2.3286958  0.7195491 -1.2294271
   1.2155651 14.412228 ]]
linear.bias: [37.279068]


In [161]:
with torch.no_grad():
    predicted = model(X_test).detach().numpy()

In [165]:
def calculate_mse(predicted, actual):

    assert len(predicted) == len(actual)

    squared_errors = np.square(predicted - actual)
    sum_squared_errors = np.sum(squared_errors)
    mse = sum_squared_errors / len(predicted)

    return mse

In [166]:
mse = calculate_mse(predicted, y_test.numpy())
print("TEST MSE:", mse)

TEST MSE: 40331.46268656717
